In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from util.fpocket import *
import nglview as nv
import re
import random
from pathlib import Path, PurePath
import ipywidgets as widgets

/home/dhruv/miniconda3/envs/dock/lib/python3.10/site-packages/nglview/__init__.py:12: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


## Identification of binding pockets with `fpocket`

In [3]:
work_dir = "yaau"

In [4]:
if not os.path.exists(f"{work_dir}/yaau_AF_out"):
    fpocket(f"{work_dir}/yaau_AF.pdb", f"{work_dir}/yaau_AF_out")

Top 5 pockets

In [5]:
pockets = parse_fpocket_info(f"{work_dir}/yaau_AF_out")
pockets = pockets.sort_values("Druggability Score", ascending=False)
pockets.head()

,Score,Druggability Score,Number of Alpha Spheres,Total SASA,Polar SASA,Apolar SASA,Volume,Mean local hydrophobic density,Mean alpha sphere radius,Mean alpha sphere solvent accessibility,Apolar alpha sphere proportion,Hydrophobicity score,Volume score,Polarity score,Charge score,Proportion of polar atoms,Alpha sphere density,Center of mass - alpha sphere max distance,Flexibility
Pocket 6,0.059,0.789,54.0,131.087,4.286,126.801,355.788,42.98,3.829,0.461,0.944,58.8,3.933,2.0,0.0,15.152,4.443,11.077,0.945
Pocket 7,0.054,0.764,93.0,201.885,49.486,152.399,634.339,47.298,3.85,0.421,0.613,26.75,4.3,12.0,2.0,30.769,5.163,12.139,0.565
Pocket 9,0.04,0.167,42.0,104.383,5.357,99.025,316.89,29.515,3.896,0.582,0.786,70.444,3.778,0.0,0.0,29.63,3.65,9.355,0.953
Pocket 11,0.034,0.154,31.0,124.804,1.071,123.732,425.753,23.241,3.92,0.593,0.935,66.182,4.364,3.0,0.0,17.857,4.861,11.551,0.97
Pocket 18,-0.042,0.108,39.0,87.612,4.286,83.326,268.723,32.0,4.199,0.603,0.846,70.0,4.571,0.0,0.0,13.636,2.18,6.497,0.899


In [6]:
# random colors for cavities
r = lambda: random.randint(0, 255)
pockets_dir = f"{work_dir}/yaau_AF_out/pockets"
path_pockets = [str(i) for i in Path(pockets_dir).iterdir()]
path_pockets_pdb = [
    str(i) for i in Path(pockets_dir).iterdir() if PurePath(i).suffix == ".pdb"
]
path_pockets_pqr = [
    str(i) for i in Path(pockets_dir).iterdir() if PurePath(i).suffix == ".pqr"
]

# determine top-5 pockets from previous 'pockets' dataframe
top5_nums = []
try:
    idxs = pockets.head(5).index.astype(str)
    for s in idxs:
        m = re.search(r"(\d+)", s)
        if m:
            top5_nums.append(m.group(1))
        elif s.isdigit():
            top5_nums.append(s)
except Exception:
    # fallback: 1..5
    top5_nums = [str(i) for i in range(1, 6)]
top5_set = set(top5_nums)

# load structure
view = nv.NGLWidget()
c = view.add_component(nv.FileStructure(f"{work_dir}/yaau_AF.pdb"))

# load cavities (c) and pockets (p) and create pocketNames list (only top-5)
c = {}
p = {}
pocket_names = []
for pock in path_pockets:
    g = re.findall("(?:pocket)(\d+)(?:_\w+)\.(\w+)", pock)
    if not g:
        continue
    i = g[0][0]
    suff = g[0][1]
    if i not in top5_set:
        continue
    if not [item for item in pocket_names if ("pocket" + i) in item]:
        pocket_names.append(("pocket" + i, int(i)))

    if suff == "pdb":
        c[i] = view.add_component(
            filename=nv.FileStructure(pock), **{"name": "pocket" + i}
        )
        c[i].clear()
    else:
        p[i] = view.add_component(
            filename=nv.FileStructure(pock), **{"name": "pocket" + i}
        )
        p[i].clear()

# sort pocket names
pocket_names.sort(key=lambda tup: tup[1])

# representation for cavities (only top-5)
for pock in path_pockets_pdb:
    g = re.findall("(?:pocket)(\d+)(?:_\w+)\.(\w+)", pock)
    if not g:
        continue
    i = g[0][0]
    if i not in top5_set:
        continue
    c[i].add_surface(
        color="#cc0000",
        radius="1.5",
        lowResolution=True,
        # 0: low resolution
        smooth=1,
        # useWorker= True,
        wrap=True,
    )

# representation for pockets (only top-5)
for pock in path_pockets_pqr:
    g = re.findall("(?:pocket)(\d+)(?:_\w+)\.(\w+)", pock)
    if not g:
        continue
    i = g[0][0]
    if i not in top5_set:
        continue
    p[i].add_surface(component=i, color="skyblue", surfaceType="av", contour=True)

view.center()
view._remote_call("setSize", target="Widget", args=["", "600px"])
view

# show pocket labels
code = """
var stage = this.stage;
var view = this.stage.viewer;
var clist_len = stage.compList.length;
var i = 0;
for(i = 0; i <= clist_len; i++){
    if(stage.compList[i] != undefined && stage.compList[i].structure != undefined && stage.compList[i].parameters.ext === 'pqr') {        

        var elm = document.createElement("div");
        elm.innerText = 'pocket' + stage.compList[i].object.name.match(/\d+/g)
        elm.style.color = "black";
        elm.style.background = "rgba(201, 149, 6, .8)";
        elm.style.padding = "8px";
        
        stage.compList[i].addAnnotation(stage.compList[i].structure.center, elm)
    }
}
"""

view._execute_js_code(code)

view

NGLWidget()

In [7]:
mdsel = widgets.Dropdown(
    options=pocket_names,
    description='Select pocket:',
    disabled=False,
)
display(mdsel)

Dropdown(description='Select pocket:', options=(('pocket6', 6), ('pocket7', 7), ('pocket9', 9), ('pocket11', 1…

In [13]:
cx, cy, cz =centroid(work_dir + f"/{work_dir}_AF_out/pockets", mdsel.value)

## Docking

Ligand preparation

In [ ]:
from meeko import MoleculePreparation
from meeko import PDBQTWriterLegacy
from rdkit import Chem
from rdkit.Chem import AllChem
from scrubber import Scrub

scrub = Scrub(
    ph_low=7.4,
    ph_high=7.4,
)

mol = Chem.MolFromSmiles(
    "C(C(C(C(C(F)(F)S(=O)(=O)O)(F)F)(F)F)(F)F)(C(C(C(F)(F)F)(F)F)(F)F)(F)F"
)
mol = Chem.AddHs(mol)
AllChem.EmbedMolecule(mol)

mk_prep = MoleculePreparation()
molsetup_list = mk_prep(mol)

molsetup = molsetup_list[0]

pdbqt_string = PDBQTWriterLegacy.write_string(molsetup)

Path(work_dir, "ad").mkdir(parents=True, exist_ok=True)
with open(f'{work_dir}/ad/ligand.pdbqt', 'w') as f:
    f.write(pdbqt_string[0])

Maybe this molecule is "too" symmetric?  O=S(=O)(O)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F


Receptor preparation

In [15]:
command = ["mk_prepare_receptor.py", "-i", f"{work_dir}_AF.pdb", "-o", "ad/prepared", "-p", "--box_center", str(cx), str(cy), str(cz), "--box_size", "24", "24", "24"]
subprocess.run(command, check=True, cwd=work_dir)

@> 3424 atoms and 1 coordinate set(s) were parsed in 0.01s.



Files written:
ad/prepared.pdbqt <-- static (i.e., rigid) receptor input file


CompletedProcess(args=['mk_prepare_receptor.py', '-i', 'yaau_AF.pdb', '-o', 'ad/prepared', '-p', '--box_center', '-1.162629629629629', '12.23062962962963', '-14.933537037037038', '--box_size', '24', '24', '24'], returncode=0)